# Constant

In [34]:
import certifi
import json
import time
import random
import psycopg2
import os

API_KEY = "f8fe2458856a3904f2a649ebf1e82a68"
PG_HOST = os.environ.get('POSTGRES_HOST','')
PG_PORT = os.environ.get('POSTGRES_PORT','')
PG_DB = os.environ.get('POSTGRES_DB','')
PG_USER = os.environ.get('POSTGRES_USER','')
PG_PASS = os.environ.get('POSTGRES_PASSWORD','')

# Common library

In [31]:
try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen


def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    time.sleep(1)
    return json.loads(data)

def connectDB():
    con = psycopg2.connect(user=PG_USER,
                       password=PG_PASS,
                       host=PG_HOST,
                       port=PG_PORT,
                       database=PG_DB)
    return con

def executeSQLParm(con, param):
    cur = con.cursor()
    cur.executemany("""INSERT INTO bar(first_name,last_name) VALUES (%(first_name)s, %(last_name)s)""", param)
    records = cur.fetchall()
    cur.close()
    return records

dbCon=connectDB()

ModuleNotFoundError: No module named 'psycopg2'

# Get symbol list

In [29]:
url = ("https://financialmodelingprep.com/api/v3/stock/list?apikey="+API_KEY)
#url =("https://financialmodelingprep.com/api/v3/stock/list?apikey=f8fe2458856a3904f2a649ebf1e82a68")
retDict=get_jsonparsed_data(url)
symbol_list = []

for item in retDict:
    symbol_list.append(item.get('symbol'))

# Save request quotas
random.shuffle(symbol_list)
symbol_list=symbol_list[0:3]
print(symbol_list)

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


NameError: name 'symbol_lsit' is not defined

# Get delisted companies

In [25]:
startpage=0
maxpage=6

for page in range(startpage,  maxpage):
    url = ("https://financialmodelingprep.com/api/v3/delisted-companies?page="+str(page)+"&apikey="+API_KEY)
    retDict=get_jsonparsed_data(url)
    if retDict:
        print(page)
        #for item in retDict:
            #print(item.get('companyName'))
    else:
        break

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


# Get historical dividends

In [26]:
for symbol in symbol_list[0:2]:
    url = ("https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/"+symbol+"?apikey="+API_KEY)
    retDict=get_jsonparsed_data(url)
    print(retDict.get('symbol'))

/home/glue_user/.local/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.


SPY
CMCSA


# Clean whatever we need to

In [ ]:
dbCon.close()